In [1]:
import pandas as pd 

import json
import h5py
import ase 
from ase import io
from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.analysis.graphs import MoleculeGraph
from pymatgen.core import Molecule, Structure

elem_to_num = {
    "H": 1,
    "He": 2,
    "Li": 3,
    "Be": 4,
    "B": 5,
    "C": 6,
    "N": 7,
    "O": 8,
    "F": 9,
    "Ne": 10,
    "Na": 11,
    "Mg": 12,
    "Al": 13,
    "Si": 14,
    "P": 15,
    "S": 16,
    "Cl": 17
}    

num_to_elem = {v: k for k, v in elem_to_num.items()}

In [2]:
df = pd.read_json("/home/santiagovargas/dev/berkeley_pes/data/test_libe.json")
file_xyz = "/home/santiagovargas/dev/SpookyNet/data/10/rad_qm9_traj_subset_train10.xyz"
statistics = "/home/santiagovargas/dev/SpookyNet/data/10/h5/statistics.json"
h5_test = '/home/santiagovargas/dev/SpookyNet/data/10/h5/train/train_0.h5'

train_chunk_5 = "/home/santiagovargas/Downloads/radqm9_65_10_25_trajectory_full_data_20240916_train_subset_0.05.xyz"
train_chunk_10 = "/home/santiagovargas/Downloads/radqm9_65_10_25_trajectory_full_data_20240916_train_subset_0.1.xyz"
train_chunk_25 = "/home/santiagovargas/Downloads/radqm9_65_10_25_trajectory_full_data_20240916_train_subset_0.25.xyz"
validation = "/home/santiagovargas/Downloads/radqm9_65_10_25_trajectory_full_data_20240916_val.xyz"
test = "/home/santiagovargas/Downloads/radqm9_65_10_25_trajectory_full_data_20240916_test.xyz"
ood = "/home/santiagovargas/Downloads/radqm9_65_10_25_trajectory_full_data_20240916_ood.xyz"



In [13]:
# read first line from ood file 
with open(ood, 'r') as f:
    first_line = f.readline()
    print(first_line)
    second_line = f.readline()
    print(second_line)

25

Properties=species:S:1:pos:R:3:mulliken_partial_charges:R:1:mulliken_partial_spins:R:1:resp_partial_charges:R:1:REF_forces:R:3 dipole_moments="2.0293 -2.345 0.8163" resp_dipole_moments="1.4966 -2.0439 0.4582" weight=128.215 REF_energy=-10584.532734931203 charge=0 spin=3 mol_id=0556a1507b6fcab91ffc49694ffc2992-C8H16O1-0-3 position_type=start relative_energy=2.070170675055124 pbc="F F F"



In [14]:
dipole_moments="2.0293 -2.345 0.8163" resp_dipole_moments="1.4966 -2.0439 0.4582"

SyntaxError: invalid syntax (1957233202.py, line 1)

In [4]:
# items needed 

In [5]:
df[["molecule", "energy", "gradient"]].iloc[0]["molecule"]
row = df.iloc[0]
spin = row["molecule"]["spin_multiplicity"]
charge = row["molecule"]["charge"]
force = row["gradient"]
energy = row["energy"]

In [6]:

pos_elem_list = [(elem_to_num[i["name"]], i["xyz"]) for i in row["molecule"]["sites"]] 
elem = [i[0] for i in pos_elem_list]
pos = [i[1] for i in pos_elem_list]

In [17]:

def convert_to_rows(file_xyz):
    atoms = io.read(file_xyz, index=':')
    list_of_rows = []
    for atom in atoms: 
        
        info = atom.info
        full_info = atom.__dict__["arrays"]
        spin = info['spin']
        charge = info['charge']
        relative_energy = info['relative_energy']
        #total_energy = info['total_energy']
        atomic_num = atom.get_atomic_numbers()
        
        #forces = atom.get_forces()
        
        pos = atom.get_positions()

        forces = full_info['REF_forces']
        resp_partial_charges = full_info['resp_partial_charges']
        mulliken_partial_charges = full_info['mulliken_partial_charges']
        mulliken_partial_spins = full_info['mulliken_partial_spins']
        dipole_moments = info['dipole_moments']
        resp_dipole_moments = info['resp_dipole_moments']
    
        dict_temp = {
            "sites": [
                {
                #'species': [{'element': num_to_elem[atomic_num[i]], 'occu': 1}],
                'xyz': pos[i],
                'name': num_to_elem[atomic_num[i]],
                'properties': {}
                }
                for i in range(len(atomic_num))
            ],
        }
        dict_temp["charge"] = charge
        dict_temp["spin_multiplicity"] = spin
        
        dict_total = {
            #"energy": total_energy,
            "gradient": forces, 
            "molecule": dict_temp,
            "relative_energy": relative_energy,
            'dipole_moments': dipole_moments,
            'resp_dipole_moments': resp_dipole_moments,
            "mulliken_partial_charges": mulliken_partial_charges,
            "mulliken_partial_spins": mulliken_partial_spins,
            "resp_partial_charges": resp_partial_charges,
            
        }
        list_of_rows.append(dict_total)
    
    # convert to dataframe 
    return pd.DataFrame(list_of_rows)

df = convert_to_rows(train_chunk_5)
# save
df.to_json("train_chunk_5_radqm9_20240916.json")

In [18]:

file_xyz = train_chunk_5
atoms = io.read(file_xyz, index=':')
list_of_rows = []
for atom in atoms: 
    
    info = atom.info
    full_info = atom.__dict__["arrays"]
    spin = info['spin']
    charge = info['charge']
    relative_energy = info['relative_energy']
    #total_energy = info['total_energy']
    
    atomic_num = atom.get_atomic_numbers()
    pos = atom.get_positions()
    #forces = atom.get_forces() # err here 
    
    forces = full_info['REF_forces']
    dipole_moments = info['dipole_moments']
    resp_partial_charges = full_info['resp_partial_charges']
    resp_dipole_moments = info['resp_dipole_moments']
    mulliken_partial_charges = full_info['mulliken_partial_charges']
    mulliken_partial_spins = full_info['mulliken_partial_spins']
    
    
    dict_temp = {
        "sites": [
            {
            #'species': [{'element': num_to_elem[atomic_num[i]], 'occu': 1}],
            'xyz': pos[i],
            'name': num_to_elem[atomic_num[i]],
            'properties': {}
            }
            for i in range(len(atomic_num))
        ],
    }
    dict_temp["charge"] = charge
    dict_temp["spin_multiplicity"] = spin
    
    dict_total = {
        "molecule": dict_temp,
        #"energy": total_energy,
        "gradient": 0, 
        "relative_energy": relative_energy,
        "mulliken_partial_charges": mulliken_partial_charges,
        "mulliken_partial_spins": mulliken_partial_spins,
        "resp_partial_charges": resp_partial_charges, 
        'dipole_moments': dipole_moments,
        'resp_dipole_moments': resp_dipole_moments
    }

    break 

full_info


{'numbers': array([7, 6, 8, 8, 6, 6, 9, 9, 9, 1, 1, 1, 1]),
 'positions': array([[-0.16878954,  1.20249863, -0.04144335],
        [-0.0987566 , -0.12505141,  0.23721661],
        [ 0.55867907, -0.63571692,  1.1090262 ],
        [-0.86547791, -0.81374151, -0.67056466],
        [-0.94176041, -2.21389414, -0.47033705],
        [-2.28648015, -2.56663258,  0.14898815],
        [-3.3061026 , -2.22827849, -0.65013098],
        [-2.35069852, -3.88858071,  0.36197702],
        [-2.46459547, -1.94311631,  1.31851042],
        [ 0.22127548,  1.83095585,  0.63868941],
        [-0.88433481,  1.54121139, -0.66111086],
        [-0.86608158, -2.70271344, -1.44397605],
        [-0.14504256, -2.55092205,  0.19495526]]),
 'mulliken_partial_charges': array([ 0.268207,  1.162587, -0.296609, -0.835803,  0.458088,  1.350123,
        -0.41522 , -0.411446, -0.428372,  0.09002 ,  0.022306, -0.136292,
         0.172412]),
 'mulliken_partial_spins': array([ 6.77695e-01, -1.48258e-01,  4.94031e-01, -4.23600e-03,
 

In [19]:
df = convert_to_rows(train_chunk_5)
# save
df.to_json("train_chunk_5_radqm9_20240916.json")

In [20]:
df = convert_to_rows(train_chunk_10)
df.to_json("train_chunk_10_radqm9_20240916.json")

In [21]:
df = convert_to_rows(train_chunk_25)
df.to_json("train_chunk_25_radqm9_20240916.json")

In [22]:
df = convert_to_rows(test)
df.to_json("test_radqm9_20240916.json")

In [23]:
df = convert_to_rows(ood)
df.to_json("ood_radqm9_20240916.json")

In [25]:
df = convert_to_rows(validation)
df.to_json("validation_radqm9_20240916.json")

In [26]:
import pandas as pd 
df = pd.read_json("train_chunk_10_radqm9_20240916.json")

In [3]:
df = pd.read_json("/home/santiagovargas/dev/berkeley_pes/data/test_libe.json")
df 

,molecule,energy,gradient,mulliken,resp
0,"{'@module': 'pymatgen.core.structure', '@class...",-314.672779,"[[-0.0082655, 0.0086199, 0.0073333], [-0.00235...","[-0.601016, 0.240001, 0.37568399999999996, -0....","[-0.544887, 0.448604, -0.122679, -0.0976500000..."
1,"{'@module': 'pymatgen.core.structure', '@class...",-314.672781,"[[-0.008185, 0.0085151, 0.0073276999999999995]...","[-0.599772, 0.240722, 0.374788, -0.270389, -0....","[-0.5439579999999999, 0.446445, -0.11959499999..."
2,"{'@module': 'pymatgen.core.structure', '@class...",-314.672785,"[[-0.0081592, 0.0084025, 0.0072723], [-0.00244...","[-0.5977800000000001, 0.242358, 0.372305, -0.2...","[-0.541744, 0.44226099999999996, -0.113802, -0..."
3,"{'@module': 'pymatgen.core.structure', '@class...",-314.672785,"[[-0.0081797, 0.0084421, 0.007256599999999999]...","[-0.5976, 0.242826, 0.371616, -0.2696300000000...","[-0.541223, 0.441698, -0.11405799999999999, -0..."
4,"{'@module': 'pymatgen.core.structure', '@class...",-382.580983,"[[-0.056095599999999995, 0.1844031, -0.1872320...","[[0.346539, -0.087966], [0.011578, 0.975015], ...","[0.580056, -0.055658, 0.220545, 0.141087, 0.14..."
...,...,...,...,...,...
95,"{'@module': 'pymatgen.core.structure', '@class...",-429.142580,"[[-0.051751899999999997, 0.015801, -0.0448554]...","[[-0.971999, 0.320648], [0.028062999999999998,...","[-1.381445, -0.12766000000000002, 0.040988, -0..."
96,"{'@module': 'pymatgen.core.structure', '@class...",-429.142581,"[[-0.0517168, 0.0159181, -0.0447986], [-0.0087...","[[-0.9749249999999999, 0.320172], [0.029715, 0...","[-1.387144, -0.12709499999999999, 0.042255, -0..."
97,"{'@module': 'pymatgen.core.structure', '@class...",-429.087265,"[[-0.0504689, 0.0223387, -0.059565099999999996...","[[-0.8351649999999999, 0.416321], [-0.09125000...","[-1.217231, -0.5378459999999999, -0.1955579999..."
98,"{'@module': 'pymatgen.core.structure', '@class...",-429.103231,"[[-0.0383129, 0.0276367, -0.047001799999999996...","[[-0.8538800000000001, 0.43920499999999996], [...","[-1.238272, -0.50309, -0.21068299999999998, 0...."


In [4]:
df = pd.read_json("train_chunk_5_radqm9_20240916.json")
df = df.head(100)
df.to_json("train_debug.json")

In [2]:
import pandas as pd 
df = pd.read_json("./data/train_debug.json")

In [3]:
df

,molecule,gradient,relative_energy,mulliken_partial_charges,mulliken_partial_spins
0,"{'sites': [{'xyz': [-0.16878954000000002, 1.20...","[[-0.25519239, 0.31389584, -1.07098322], [2.15...",-16884.120080,"[0.268207, 1.162587, -0.296609, -0.835803, 0.4...","[0.6776949999999999, -0.148258, 0.494031, -0.0..."
1,"{'sites': [{'xyz': [0.04108741, 1.41247724, -0...","[[0.03472533, 0.26830502, -0.00347099], [-0.04...",-15511.441376,"[0.249433, -0.180882, -0.969652, -0.831611, -0...","[0.8098719999999999, -0.679469, 0.113456999999..."
2,"{'sites': [{'xyz': [0.040048, 1.41562075, -0.1...","[[-0.02891978, 0.08230105, 0.02377243], [0.019...",-15511.462511,"[0.272173, -0.193937, -0.9694919999999999, -0....","[0.755562, -0.641898, 0.074734, 1.984316999999..."
3,"{'sites': [{'xyz': [0.00720869, 1.42677097, -0...","[[0.00127527, -0.00134212, -0.00030853], [0.00...",-15511.477075,"[0.285389, -0.169006, -1.178167, -0.973846, -0...","[0.737545, -0.57165, 0.140509, 2.01673, 0.2360..."
4,"{'sites': [{'xyz': [0.09588533, 1.357197670000...","[[0.00212887, -0.00297734, -0.00779045], [0.00...",-15503.667940,"[1.029559, -0.7794800000000001, -0.75521699999...","[0.0024579999999999997, 0.676882, -0.014731, 0..."
...,...,...,...,...,...
95,"{'sites': [{'xyz': [0.18940762, 1.5651586, -0....","[[0.03010763, 0.23336885000000002, -0.0813086]...",-14501.924265,"[-0.8347129999999999, -0.567639, 0.57397000000...","[2.004691, 0.13386299999999998, 0.280406999999..."
96,"{'sites': [{'xyz': [0.19248559, 1.56691958, -0...","[[0.04855788, 0.09186041, -0.01159568], [-0.18...",-14501.944435,"[-0.86975, -0.586509, 0.548312, -0.15795599999...","[2.002046, 0.136766, 0.270106, 0.010808, 0.441..."
97,"{'sites': [{'xyz': [0.16701757, 1.564234470000...","[[-0.00573356, -0.00111586, -0.002545390000000...",-14501.958398,"[-0.9392619999999999, -0.5559149999999999, 0.5...","[1.977856, 0.123932, 0.288742, 0.009842, 0.462..."
98,"{'sites': [{'xyz': [0.19303387, 1.591896859999...","[[0.00403149, 0.00163522, -0.0003291], [-0.004...",-14494.589037,"[0.837839, -0.424774, 0.9763069999999999, -0.2...","[-0.0461, 0.459459, -0.081068, 0.572418, -0.06..."
